# 🔬 Part 2: Training the Delta Observer

This notebook trains the **Delta Observer** network that learns to map between monolithic and compositional representations.

## Architecture

```
Monolithic (64D) ──→ Encoder ──┐
                                ├──→ Shared Latent (16D) ──→ Decoders
Compositional (64D) ─→ Encoder ──┘
```

The 16D latent space learns the **semantic primitive** that distinguishes these representations.

📄 **Paper:** [OSF MetaArXiv](https://doi.org/10.17605/OSF.IO/CNJTP)  
🔗 **Code:** [github.com/EntroMorphic/delta-observer](https://github.com/EntroMorphic/delta-observer)

---

**Note:** For the complete **Online Delta Observer** pipeline (recommended), use **`99_full_reproduction.ipynb`** which trains all models concurrently and captures the transient clustering phenomenon. This notebook demonstrates the post-hoc training approach.

---

## 📦 Setup

In [ ]:
# Install dependencies if needed (Colab)
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

install_if_needed('torch')
install_if_needed('matplotlib')
install_if_needed('scikit-learn')

print('✅ Dependencies ready!')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️ Using device: {device}')

# Plotting style
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Colors
COLORS = {
    'train': '#3498db',
    'val': '#e74c3c',
    'accent': '#2ecc71'
}

---

## 📂 Load Pre-computed Activations

In [ ]:
# Clone repository if running in Colab
repo_dir = 'delta-observer'
if not os.path.exists(repo_dir) and not os.path.exists('../data'):
    print('📥 Cloning delta-observer repository...')
    !git clone https://github.com/EntroMorphic/delta-observer.git
    print('✅ Repository cloned!')

# Smart path detection
possible_data_dirs = ['../data', 'data', 'delta-observer/data']
data_dir = next((p for p in possible_data_dirs if os.path.exists(p)), None)

possible_models_dirs = ['../models', 'models', 'delta-observer/models']
models_dir = next((p for p in possible_models_dirs if os.path.exists(p)), '../models')

os.makedirs(models_dir, exist_ok=True)
os.makedirs('../figures', exist_ok=True)

print(f'📁 Data directory: {data_dir}')
print(f'📁 Models directory: {models_dir}')

In [ ]:
# Load activations
mono_data = np.load(os.path.join(data_dir, 'monolithic_activations.npz'))
comp_data = np.load(os.path.join(data_dir, 'compositional_activations.npz'))

mono_activations = mono_data['activations']
comp_activations = comp_data['activations']
inputs = mono_data['inputs']
carry_counts = mono_data['carry_counts']

print(f'📊 Monolithic activations: {mono_activations.shape}')
print(f'📊 Compositional activations: {comp_activations.shape}')
print(f'📊 Carry counts: {np.bincount(carry_counts)}')

In [ ]:
# 🎨 Visualize input activations
from sklearn.decomposition import PCA

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, activations, name in [
    (axes[0], mono_activations, 'Monolithic'),
    (axes[1], comp_activations, 'Compositional')
]:
    pca = PCA(n_components=2)
    act_2d = pca.fit_transform(activations)
    scatter = ax.scatter(act_2d[:, 0], act_2d[:, 1], c=carry_counts, 
                         cmap='viridis', s=30, alpha=0.7)
    ax.set_title(f'{name} Activations (PCA)', fontsize=12, fontweight='bold')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')

plt.colorbar(scatter, ax=axes, label='Carry Count', shrink=0.8)
plt.suptitle('🔍 Input Activations to Delta Observer', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\n💡 These are the representations the Delta Observer will learn to bridge!')

---

## 📋 Dataset Class

In [ ]:
class DeltaObserverDataset(Dataset):
    def __init__(self, mono_act, comp_act, carry_counts, inputs):
        self.mono_act = torch.tensor(mono_act, dtype=torch.float32)
        self.comp_act = torch.tensor(comp_act, dtype=torch.float32)
        self.carry_counts = torch.tensor(carry_counts, dtype=torch.long)
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
    
    def __len__(self):
        return len(self.mono_act)
    
    def __getitem__(self, idx):
        return {
            'mono_act': self.mono_act[idx],
            'comp_act': self.comp_act[idx],
            'carry_count': self.carry_counts[idx],
            'input': self.inputs[idx],
        }

dataset = DeltaObserverDataset(mono_activations, comp_activations, carry_counts, inputs)
print(f'📊 Dataset size: {len(dataset)}')

# Split 80/20
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f'📊 Train: {len(train_dataset)}, Val: {len(val_dataset)}')

---

## 🏗️ Delta Observer Architecture

In [ ]:
class DeltaObserver(nn.Module):
    def __init__(self, mono_dim=64, comp_dim=64, latent_dim=16):
        super().__init__()
        
        # Dual encoders
        self.mono_encoder = nn.Sequential(
            nn.Linear(mono_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
        )
        
        self.comp_encoder = nn.Sequential(
            nn.Linear(comp_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
        )
        
        # Shared latent encoder
        self.shared_encoder = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, latent_dim),
        )
        
        # Decoders
        self.mono_decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, mono_dim),
        )
        
        self.comp_decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, comp_dim),
        )
        
        # Carry count regressor
        self.carry_regressor = nn.Sequential(
            nn.Linear(latent_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
        )
        
        self.latent_dim = latent_dim
    
    def encode(self, mono_act, comp_act):
        mono_enc = self.mono_encoder(mono_act)
        comp_enc = self.comp_encoder(comp_act)
        joint = torch.cat([mono_enc, comp_enc], dim=-1)
        return self.shared_encoder(joint)
    
    def forward(self, mono_act, comp_act):
        latent = self.encode(mono_act, comp_act)
        mono_recon = self.mono_decoder(latent)
        comp_recon = self.comp_decoder(latent)
        carry_pred = self.carry_regressor(latent)
        
        return {
            'latent': latent,
            'mono_recon': mono_recon,
            'comp_recon': comp_recon,
            'carry_pred': carry_pred,
        }

model = DeltaObserver(
    mono_dim=mono_activations.shape[1], 
    comp_dim=comp_activations.shape[1], 
    latent_dim=16
).to(device)

print(f'🏗️ Delta Observer parameters: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# 🎨 Visualize architecture
fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('off')

# Draw components
components = [
    # (x, y, width, height, label, color)
    (0.05, 0.7, 0.12, 0.15, f'Mono\n({mono_activations.shape[1]}D)', '#e74c3c'),
    (0.05, 0.3, 0.12, 0.15, f'Comp\n({comp_activations.shape[1]}D)', '#3498db'),
    (0.25, 0.7, 0.12, 0.1, 'Encoder\n(32D)', '#9b59b6'),
    (0.25, 0.35, 0.12, 0.1, 'Encoder\n(32D)', '#9b59b6'),
    (0.45, 0.45, 0.15, 0.2, 'Shared\nLatent\n(16D)', '#f39c12'),
    (0.7, 0.7, 0.12, 0.1, 'Decoder', '#2ecc71'),
    (0.7, 0.35, 0.12, 0.1, 'Decoder', '#2ecc71'),
    (0.9, 0.7, 0.08, 0.1, f'Mono\nRecon', '#e74c3c'),
    (0.9, 0.35, 0.08, 0.1, f'Comp\nRecon', '#3498db'),
]

for x, y, w, h, label, color in components:
    rect = plt.Rectangle((x, y), w, h, facecolor=color, edgecolor='black', linewidth=2, alpha=0.7)
    ax.add_patch(rect)
    ax.text(x + w/2, y + h/2, label, ha='center', va='center', fontsize=9, fontweight='bold')

# Draw arrows
arrows = [
    (0.17, 0.775, 0.25, 0.75),
    (0.17, 0.375, 0.25, 0.40),
    (0.37, 0.75, 0.45, 0.55),
    (0.37, 0.40, 0.45, 0.55),
    (0.60, 0.55, 0.70, 0.75),
    (0.60, 0.55, 0.70, 0.40),
    (0.82, 0.75, 0.90, 0.75),
    (0.82, 0.40, 0.90, 0.40),
]

for x1, y1, x2, y2 in arrows:
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))

ax.set_xlim(0, 1)
ax.set_ylim(0.2, 0.95)
ax.set_title('🏗️ Delta Observer Architecture', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---

## 🏋️ Training

In [ ]:
epochs = 100
lr = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

history = {'train_loss': [], 'val_loss': [], 'train_r2': [], 'val_r2': []}
best_val_loss = float('inf')

print('🏋️ Training Delta Observer...\n')

pbar = tqdm(range(epochs), desc='Training')
for epoch in pbar:
    # Training
    model.train()
    train_loss = 0
    train_preds, train_targets = [], []
    
    for batch in train_loader:
        mono_act = batch['mono_act'].to(device)
        comp_act = batch['comp_act'].to(device)
        carry_count = batch['carry_count'].to(device).float()
        
        optimizer.zero_grad()
        outputs = model(mono_act, comp_act)
        
        # Losses
        recon_loss = F.mse_loss(outputs['mono_recon'], mono_act) + F.mse_loss(outputs['comp_recon'], comp_act)
        carry_loss = F.mse_loss(outputs['carry_pred'].squeeze(), carry_count)
        
        loss = recon_loss + carry_loss
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_preds.extend(outputs['carry_pred'].squeeze().detach().cpu().numpy())
        train_targets.extend(carry_count.cpu().numpy())
    
    train_loss /= len(train_loader)
    train_r2 = 1 - np.sum((np.array(train_preds) - np.array(train_targets))**2) / np.sum((np.array(train_targets) - np.mean(train_targets))**2)
    
    # Validation
    model.eval()
    val_loss = 0
    val_preds, val_targets = [], []
    
    with torch.no_grad():
        for batch in val_loader:
            mono_act = batch['mono_act'].to(device)
            comp_act = batch['comp_act'].to(device)
            carry_count = batch['carry_count'].to(device).float()
            
            outputs = model(mono_act, comp_act)
            
            recon_loss = F.mse_loss(outputs['mono_recon'], mono_act) + F.mse_loss(outputs['comp_recon'], comp_act)
            carry_loss = F.mse_loss(outputs['carry_pred'].squeeze(), carry_count)
            
            loss = recon_loss + carry_loss
            val_loss += loss.item()
            
            val_preds.extend(outputs['carry_pred'].squeeze().cpu().numpy())
            val_targets.extend(carry_count.cpu().numpy())
    
    val_loss /= len(val_loader)
    val_r2 = 1 - np.sum((np.array(val_preds) - np.array(val_targets))**2) / np.sum((np.array(val_targets) - np.mean(val_targets))**2)
    
    scheduler.step()
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), os.path.join(models_dir, 'delta_observer_best.pt'))
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_r2'].append(train_r2)
    history['val_r2'].append(val_r2)
    
    pbar.set_postfix({'Loss': f'{val_loss:.4f}', 'R²': f'{val_r2:.4f}'})

print(f'\n✅ Best Val Loss: {best_val_loss:.4f}')

In [ ]:
# 🎨 Visualize training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax1 = axes[0]
ax1.plot(history['train_loss'], label='Train', color=COLORS['train'], linewidth=2)
ax1.plot(history['val_loss'], label='Validation', color=COLORS['val'], linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('📉 Training Loss', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# R² curves
ax2 = axes[1]
ax2.plot(history['train_r2'], label='Train', color=COLORS['train'], linewidth=2)
ax2.plot(history['val_r2'], label='Validation', color=COLORS['val'], linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('R² (Carry Prediction)', fontsize=12)
ax2.set_title('📈 Linear Accessibility (R²)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig('../figures/delta_observer_training.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 Final R² (Validation): {history["val_r2"][-1]:.4f}')

---

## 🔍 Extract and Analyze Latent Space

In [ ]:
# Load best model and extract latents
model.load_state_dict(torch.load(os.path.join(models_dir, 'delta_observer_best.pt')))
model.eval()

full_loader = DataLoader(dataset, batch_size=64, shuffle=False)
all_latents = []
all_carry = []

with torch.no_grad():
    for batch in full_loader:
        latent = model.encode(batch['mono_act'].to(device), batch['comp_act'].to(device))
        all_latents.append(latent.cpu().numpy())
        all_carry.append(batch['carry_count'].numpy())

latent_space = np.concatenate(all_latents)
carry_counts_all = np.concatenate(all_carry)

print(f'📊 Latent space: {latent_space.shape}')

In [ ]:
# Compute metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, silhouette_score

# R² via linear regression
reg = LinearRegression().fit(latent_space, carry_counts_all)
r2 = r2_score(carry_counts_all, reg.predict(latent_space))

# Silhouette score
sil = silhouette_score(latent_space, carry_counts_all)

print('\n' + '='*60)
print('📊 DELTA OBSERVER LATENT SPACE METRICS')
print('='*60)
print(f'\n   R² (Linear Accessibility):     {r2:.4f}')
print(f'   Silhouette (Clustering):       {sil:.4f}')
print('\n' + '='*60)

In [ ]:
# 🎨 Visualize latent space
from sklearn.decomposition import PCA

try:
    from umap import UMAP
    reducer = UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    method_name = 'UMAP'
except ImportError:
    reducer = PCA(n_components=2, random_state=42)
    method_name = 'PCA'

latent_2d = reducer.fit_transform(latent_space)

fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(latent_2d[:, 0], latent_2d[:, 1], c=carry_counts_all, 
                     cmap='viridis', s=50, alpha=0.7, edgecolors='white', linewidth=0.5)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Carry Count', fontsize=12)
cbar.set_ticks([0, 1, 2, 3, 4])

# Add metrics box
textstr = f'R² = {r2:.4f}\nSilhouette = {sil:.4f}'
props = dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='gray')
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props, fontweight='bold')

ax.set_xlabel(f'{method_name} Dimension 1', fontsize=12)
ax.set_ylabel(f'{method_name} Dimension 2', fontsize=12)
ax.set_title('🔬 Delta Observer Latent Space\n(Colored by Carry Count)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/delta_latent_space.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save latent space
np.savez(os.path.join(data_dir, 'delta_latent_umap.npz'),
         latents=latent_space,
         carry_counts=carry_counts_all,
         bit_positions=np.zeros_like(carry_counts_all))  # Placeholder

print(f'✅ Latent space saved to {os.path.join(data_dir, "delta_latent_umap.npz")}')

---

## 📝 Summary

| Metric | Value | Interpretation |
|--------|-------|----------------|
| **R² (Linear Accessibility)** | ~0.95 | Semantic info is linearly accessible |
| **Silhouette (Clustering)** | ~0.03 | No geometric clustering |

**Key Finding:** The post-hoc Delta Observer achieves high linear accessibility but lower than the Online Observer (0.9505 vs 0.9879).

**Why?** The Online Observer captures temporal information during training that post-hoc analysis misses!

---

## 🚀 Next Steps

Continue to **`03_analysis_visualization.ipynb`** for deeper geometric analysis.

| Notebook | Description | Colab |
|----------|-------------|-------|
| **03_analysis_visualization** | Geometric analysis | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EntroMorphic/delta-observer/blob/main/notebooks/03_analysis_visualization.ipynb) |
| **99_full_reproduction** | Complete Online pipeline | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EntroMorphic/delta-observer/blob/main/notebooks/99_full_reproduction.ipynb) |

---

**For Science!** 🔬🌊